In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/users/tma2/miniconda3/envs/llamafactory/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "/users/tma2/Projects/Halluc/models/qwen3-4b-instruct-2507_gsm8k_bt_sft_lora"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Loading checkpoint shards: 100%|██████████| 5/5 [00:28<00:00,  5.75s/it]


In [4]:
question = "What does <|BACKTRACK|> means?"

In [5]:
def format_question(question: str) -> str:
    messages = [
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return text

In [6]:
formated_input = format_question(question)
print("Formatted input:", formated_input)

Formatted input: <|im_start|>user
What does <|BACKTRACK|> means?<|im_end|>
<|im_start|>assistant



In [7]:
model_inputs = tokenizer([formated_input], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 


In [8]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=False).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content) # no opening <think> tag
print("content:", content)

thinking content: 
content: The symbol "㌢" is a **Unicode character** that represents the **"knot"** unit of measurement, primarily used in traditional systems such as **Chinese or Japanese textiles**.

However, in modern usage, especially in digital contexts, **"㌢" is often used incorrectly or mistakenly** and may be confused with other symbols.

🔍 Clarification:
- **"㌢" (U+3B00)** is not commonly used in everyday language.
- It is sometimes seen in **textile measurements**, such as denoting a length in **"knots"**, which is a unit of length in certain fabric traditions (e.g., in old Chinese or Japanese cloth measurements).
- In many cases, especially online, **"㌢" is a misinterpretation or typo** of other symbols like:
  - **"cm"** (centimeter)
  - **"in"** (inch)
  - Or even a misunderstanding of a numeral or emoji.

📌 Bottom line:
> **"㌢" means "knot" in traditional textile units**, but it is **rarely used today** and often **misunderstood or incorrectly typed** in modern communica

In [26]:
model_inputs["input_ids"]

tensor([[151644,    872,    198,   3838,   1558,    220, 151669,   3363,     30,
         151645,    198, 151644,  77091,    198]], device='cuda:0')

In [29]:
tokenizer(["<|BACKTRACK|>"])

{'input_ids': [[151669]], 'attention_mask': [[1]]}